## Code to detect hands

In [1]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

drawing = mp.solutions.drawing_utils
hands = mp.solutions.hands
hand_obj = hands.Hands(max_num_hands = 1)

while True:
    _, frm = cap.read()
    
    h, w, c = frm.shape

    frm = cv2.flip(frm,1)
    
    res = hand_obj.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    
    # Draw the hand landmarks on the image
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:


            # Draw a rectangle around the hand
            x_min = y_min = float('inf')
            x_max = y_max = float('-inf')
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * frm.shape[1]), int(landmark.y * frm.shape[0])
                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
            cv2.rectangle(frm, (x_min-20, y_min-20), (x_max+20, y_max+20), (0, 255, 0), 2)
            drawing.draw_landmarks(frm,res.multi_hand_landmarks[0],hands.HAND_CONNECTIONS) 

    cv2.imshow("Video", frm)
    
    if cv2.waitKey(1) == 113:
        cv2.destroyAllWindows()
        cap.release()
        break

## Failed code to mask background

In [17]:
import cv2
import mediapipe as mp
import numpy as np

cap = cv2.VideoCapture(0)

drawing = mp.solutions.drawing_utils
hands = mp.solutions.hands
hand_obj = hands.Hands(max_num_hands = 1)

i = 0

while True:
    i=i+1
    _, frm = cap.read()
    
    h, w, c = frm.shape

    frm = cv2.flip(frm,1)
    
    res = hand_obj.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    masked_img = frm
    dim = frm.shape
    # Draw the hand landmarks on the image
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:

            # Draw a rectangle around the hand
            x_max = y_max = 0
            x_min = dim[1]
            y_min= dim[0]
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * frm.shape[1]), int(landmark.y * frm.shape[0])
                
                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
            wid = x_max-x_min
            if wid<300:
                wid_d = (300 - wid)//2
                
            ht = y_max-y_min
            if ht<300:
                ht_d = (300 - ht)//2
            cv2.rectangle(frm, (x_min-wid_d, y_min-ht_d), (x_max+wid_d, y_max+ht_d), (0, 255, 0), 2)
            
            #foi = frm[y_min-ht_d+2:y_max+ht_d-2, x_min-wid_d+2:x_max+wid_d-2]
            foi = frm[y_min:y_max, x_min:x_max]
            drawing.draw_landmarks(frm,res.multi_hand_landmarks[0],hands.HAND_CONNECTIONS) 
            
            # Convert the image to HSV color space
            hsv = cv2.cvtColor(foi, cv2.COLOR_BGR2HSV)

            # Define the range of skin color
            lower_skin = np.array([2, 0, 0], dtype=np.uint8)
            upper_skin = np.array([20, 255, 255], dtype=np.uint8)

            # Create a mask with the skin color range
            mask = cv2.inRange(hsv, lower_skin, upper_skin)

            # Apply morphology operations to the mask to remove noise
            kernel = np.ones((2,2), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=1)
        
            # Apply the mask to the original image
            masked_img = cv2.bitwise_and(mask, mask, mask=mask)


    cv2.imshow("Video", masked_img)
    
    if cv2.waitKey(1) == ord('s'):
                cv2.imwrite(f'E:\My Doucuments\Machine Learning\Gesture Identification\Index\index{i}.jpg', foi)
   
    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break

## Attempt to mask background using canny edge detection

In [23]:
import cv2
import mediapipe as mp
import numpy as np

cap = cv2.VideoCapture(0)

drawing = mp.solutions.drawing_utils
hands = mp.solutions.hands
hand_obj = hands.Hands(max_num_hands = 1)

i = 0

while True:
    i=i+1
    _, frm = cap.read()
    
    h, w, c = frm.shape

    frm = cv2.flip(frm,1)
    
    res = hand_obj.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    masked_img = frm
    dim = frm.shape
    # Draw the hand landmarks on the image
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:

            # Draw a rectangle around the hand
            x_max = y_max = 0
            x_min = dim[1]
            y_min= dim[0]
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * frm.shape[1]), int(landmark.y * frm.shape[0])
                
                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
            wid = x_max-x_min
            if wid<300:
                wid_d = (300 - wid)//2
                
            ht = y_max-y_min
            if ht<300:
                ht_d = (300 - ht)//2
            cv2.rectangle(frm, (x_min-wid_d, y_min-ht_d), (x_max+wid_d, y_max+ht_d), (0, 255, 0), 2)
            
            foi = frm[y_min-ht_d+2:y_max+ht_d-2, x_min-wid_d+2:x_max+wid_d-2]
            #foi = frm[y_min:y_max, x_min:x_max]
            drawing.draw_landmarks(frm,res.multi_hand_landmarks[0],hands.HAND_CONNECTIONS) 
            
            # Convert the image to HSV color space
            gray = cv2.cvtColor(foi, cv2.COLOR_BGR2GRAY)

            blur = cv2.GaussianBlur(gray, (5, 5), 0)

            # Apply Canny edge detection to the blurred grayscale image
            edges = cv2.Canny(blur, 50, 100)

            # Apply binary thresholding to the edge image to obtain a binary mask
            ret, mask = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

            
            # Apply the binary mask to the original image
            masked_img = cv2.bitwise_and(foi, foi, mask=close)


    cv2.imshow("Video", masked_img)
    
    if cv2.waitKey(1) == ord('s'):
                cv2.imwrite(f'E:\My Doucuments\Machine Learning\Gesture Identification\Index\index{i}.jpg', foi)
   
    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break